In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import csv
import numpy as np
import tensorflow as tf
from sklearn import metrics 

BASE_DIR = '../../../'
import sys
sys.path.append(BASE_DIR)

# custom code
import utils.utils
CONFIG = utils.utils.load_config("../../config.json")

import utils.metrics
import utils.record

Using TensorFlow backend.


In [6]:
DATASET = os.path.basename(os.getcwd()) # name of folder this file is in
RANDOM_SEED = CONFIG['random_seed']
EPOCHS = CONFIG["experiment_configs"][DATASET]["epochs"]
BATCH_SIZE = CONFIG["experiment_configs"][DATASET]["batch_size"]
IMAGE_X_SIZE = CONFIG["experiment_configs"][DATASET]["image_x_size"]
IMAGE_Y_SIZE = CONFIG["experiment_configs"][DATASET]["image_y_size"]
IMAGE_SIZE = (IMAGE_Y_SIZE, IMAGE_X_SIZE)
VAL_FULL_SPLIT = CONFIG['experiment_configs'][DATASET]['val_full_split']
HYPER_VAL_SPLIT = CONFIG['experiment_configs'][DATASET]['hyper_val_split']

print(DATASET, RANDOM_SEED, VAL_FULL_SPLIT)

# folders for processed, models
DATA_F = os.path.join(BASE_DIR, f"data/{DATASET}/")
PROCESSED_DIR = os.path.join(BASE_DIR, f'processed/{DATASET}/rs={RANDOM_SEED}/vs={VAL_FULL_SPLIT}')
MODELS_DIR = os.path.join(BASE_DIR, f'models/{DATASET}/rs={RANDOM_SEED}/vs={VAL_FULL_SPLIT}')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')

BASE_MODEL_SAVEPATH = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt="base") # mt = model_type

# base model saved here
if not os.path.exists(BASE_MODEL_SAVEPATH):
    print(f"warning: no model has been run for rs={RANDOM_SEED}")

adience_ablation 15 0.5


In [7]:
os.makedirs(RESULTS_DIR, exist_ok=True)

In [8]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

hyper_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "hyper_val"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(PROCESSED_DIR, "test"),
    batch_size=BATCH_SIZE,
    image_size=IMAGE_SIZE,
    label_mode='categorical',
    follow_links=True,
    seed = RANDOM_SEED,
)

Found 1992 files belonging to 2 classes.
Using 996 files for training.
Found 1992 files belonging to 2 classes.
Using 996 files for validation.
Found 1992 files belonging to 2 classes.


In [9]:
'''
This will standardize the pixel data
'''
def preprocess(imgs, labels):
    # turn from <0..255> to <0..1>
    imgs = imgs / 255.0
    means = np.array( [0.5, 0.5, 0.5] )
    stds = np.array( [0.5, 0.5, 0.5] )
    imgs = (imgs - means) / stds
    return imgs, labels

In [10]:
val_ds = val_ds.map(preprocess)
hyper_val_ds = hyper_val_ds.map(preprocess)
test_ds = test_ds.map(preprocess)

In [11]:
model = utils.utils.make_resnet(
    depth=2,
    random_state=RANDOM_SEED,
    input_shape=(*IMAGE_SIZE, 3),
    nc=2,
)

In [12]:
model_types = [
    "base",
    "ft",
]

metric_list = [
    'Accuracy',
]

In [15]:
os.makedirs(RESULTS_DIR, exist_ok=True)
savepath = os.path.join(RESULTS_DIR, f"results_ablation_vs={VAL_FULL_SPLIT}.csv")
saver = utils.record.Results_Recorder(savepath, DATASET)

Results file does not exist, creating it...


In [16]:
for mt in model_types:
    print(f"Model Type: {mt}")
    modelpath = utils.utils.get_savepath(MODELS_DIR, DATASET, ".h5", mt=mt)
    model.load_weights(modelpath)
    
    val_preds, val_labels = utils.utils.compute_preds(model, val_ds)
    hyper_val_preds, hyper_val_labels = utils.utils.compute_preds(model, hyper_val_ds)
    test_preds, test_labels = utils.utils.compute_preds(model, test_ds)

    val_preds = np.argmax(val_preds, axis=1)
    hyper_val_preds = np.argmax(hyper_val_preds, axis=1)
    test_preds = np.argmax(test_preds, axis=1)

    val_conf = metrics.confusion_matrix(val_labels, val_preds, normalize='all')
    hyper_val_conf = metrics.confusion_matrix(hyper_val_labels, hyper_val_preds, normalize='all')
    test_conf = metrics.confusion_matrix(test_labels, test_preds, normalize='all')
    
    for metric in metric_list:
        val_score = utils.metrics.eval_metric(val_conf, metric)
        hyper_val_score = utils.metrics.eval_metric(hyper_val_conf, metric)
        test_score = utils.metrics.eval_metric(test_conf, metric)
        
        saver.save(RANDOM_SEED, metric, mt, val_score, hyper_val_score, test_score, None)

  0%|          | 0/32 [00:00<?, ?it/s]

Model Type: base


  0%|          | 0/32 [00:00<?, ?it/s]

Model Type: ft


100%|██████████| 63/63 [00:13<00:00,  4.79it/s]


In [17]:
saver.close()